# Imports

In [1]:
import torch
import random
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import time
import copy
from torch.utils.data import Dataset, DataLoader
from torchvision.datasets import ImageFolder
import tqdm
import torch.nn.functional as F
import torchvision.utils as vutils
import matplotlib.pyplot as plt
import time
import os
import copy
from torch.utils.tensorboard import SummaryWriter
from torchvision.models import ResNet152_Weights
import torch_directml

# Helper functions

In [2]:
def plot_classes_preds(images, labels, preds, probs):
    # plot the images in the batch, along with predicted and true labels
    fig = plt.figure(figsize=(15, 5))
    for idx in np.arange(4):
        ax = fig.add_subplot(1, 4, idx + 1, xticks=[], yticks=[])
        plt.imshow(np.transpose(images[idx].T.cpu().numpy(), (1, 0, 2)))  # because is a tensor
        ax.set_title("{0}, {1:.1f}%\n(label: {2})".format(
            preds[idx],
            probs[idx] * 100.0,
            labels[idx]),
            color=("green" if preds[idx] == labels[idx].item() else "red"))
    return fig

### Loading the train dataset

In [3]:
train_transform = transforms.Compose([
    #TODO:think a better transformation pipeline
    #naive transformation
    transforms.Resize((64, 64)),
    transforms.ToTensor()
])

train_dir = './dataset/GTSRB/train'

train_dataset = datasets.ImageFolder(train_dir, train_transform)
train_size = len(train_dataset)
class_names = train_dataset.classes

print('Train size:', train_size)
print('Class names:', class_names)

Train size: 39209
Class names: ['00000', '00001', '00002', '00003', '00004', '00005', '00006', '00007', '00008', '00009', '00010', '00011', '00012', '00013', '00014', '00015', '00016', '00017', '00018', '00019', '00020', '00021', '00022', '00023', '00024', '00025', '00026', '00027', '00028', '00029', '00030', '00031', '00032', '00033', '00034', '00035', '00036', '00037', '00038', '00039', '00040', '00041', '00042']


### Loading the test dataset

In [4]:
test_transform = transforms.Compose([
    #TODO:think a better transformation pipeline
    #naive transformation
    transforms.Resize((64, 64)),
    transforms.ToTensor()
])

test_dir = './dataset/GTSRB/test'

test_dataset = datasets.ImageFolder(test_dir, test_transform)
test_size = len(test_dataset)
class_names = test_dataset.classes

print('Test size:', train_size)
print('Class names:', class_names)

Test size: 39209
Class names: ['00000', '00001', '00002', '00003', '00004', '00005', '00006', '00007', '00008', '00009', '00010', '00011', '00012', '00013', '00014', '00015', '00016', '00017', '00018', '00019', '00020', '00021', '00022', '00023', '00024', '00025', '00026', '00027', '00028', '00029', '00030', '00031', '00032', '00033', '00034', '00035', '00036', '00037', '00038', '00039', '00040', '00041', '00042']


# Defining the training phase

In [5]:
def train_model(device, model, criterion, optimizer, scheduler, train_loader, num_epochs=25):
    since = time.time()
    train_total_steps = len(train_loader)

    best_model_wts = copy.deepcopy(model.state_dict())

    for epoch in range(num_epochs):
        print('-' * 10)
        print('Epoch {}/{}'.format(epoch + 1, num_epochs))
        print('-' * 10)

        model.train()  # Set model to training mode

        running_loss = 0.0
        running_corrects = 0
        data_loader = train_loader

        for i, (images, labels) in enumerate(data_loader):
            images = images.to(device)
            labels = labels.to(device)

            # Zero the parameter gradients
            optimizer.zero_grad()

            # Forward
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            probs = [F.softmax(el, dim=0)[i].item() for i, el in zip(preds, outputs)]
            loss = criterion(outputs, labels)

            # Backward + optimize only if in training phase
            loss.backward()
            optimizer.step()

            # Statistics
            running_loss += loss.item() * images.size(0)
            running_corrects += torch.sum(preds == labels.data)

            # Calculate entropy with epsilon
            softmax_outputs = F.softmax(outputs, dim=1)
            epsilon = 1e-10  # Small epsilon value to avoid zero probabilities
            entropy = -torch.sum(softmax_outputs * torch.log2(softmax_outputs + epsilon), dim=1).mean()

            # Log scalars
            writer.add_scalar('training loss',
                              loss.item(),
                              epoch * len(train_loader) + i)
            writer.add_scalar('entropy',
                              entropy.item(),
                              epoch * len(train_loader) + i)
            writer.add_scalar('learning rate',
                              np.array(scheduler.get_last_lr()),
                              epoch * len(train_loader) + i)

            #prints the stats every 20 steps (20 batches performed)
            if (i + 1) % int(train_total_steps / 8) == 0:
                print(f'Epoch [{epoch + 1}/{num_epochs}], Step [{i + 1}/{train_total_steps}], Loss: {loss.item():.4f}')

                # Log image predictions
                selected_indices = random.sample(range(len(images)), 4)  # Select 4 random indices
                selected_images = images[selected_indices]
                selected_labels = labels[selected_indices]
                selected_preds = preds[selected_indices]
                selected_probs = [probs[i] for i in selected_indices]
                writer.add_figure('predictions vs. actuals',
                                  plot_classes_preds(selected_images, selected_labels, selected_preds, selected_probs),
                                  global_step=epoch * len(train_loader) + i)

        epoch_loss = running_loss / len(data_loader.dataset)
        epoch_acc = running_corrects.double() / len(data_loader.dataset)

        print('{} Epoch {} Loss: {:.4f} Acc: {:.4f}'.format(
            'Train phase - ', epoch + 1, epoch_loss, epoch_acc))

        scheduler.step()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))

    # Load best model weights
    model.load_state_dict(best_model_wts)
    return model


def create_dynamic_network(num_features, num_classes, num_layers=0, num_neurons=1):
    layers = []
    # Input layer to first hidden layer
    if num_layers > 0:
        layers.append(nn.Linear(num_features, num_neurons))
        layers.append(nn.ReLU())

    # Additional hidden layers
    for _ in range(1, num_layers):
        layers.append(nn.Linear(num_neurons, num_neurons))
        layers.append(nn.ReLU())

    # Always include the final specified layer
    layers.append(nn.Linear(num_neurons if num_layers > 0 else num_features, num_classes))

    return nn.Sequential(*layers)

## Training Setup

In [6]:
# default `log_dir` is "runs" - we'll be more specific here
writer = SummaryWriter('runs/TSR-SGD')

# Setting device for the computation
#device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = torch_directml.device()

# Hyperparameters
num_epochs = 32
batch_size = 100
learning_rate = 0.05
step_size = 2  # After how many epochs to apply the decay rate
decay_rate = 0.9  # new_lr = Decay rate * learning rate

num_layers = 1
num_neurons = 100

### Setting up the model using ResNet152 as backbone

In [7]:
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Model initialization
model = torchvision.models.resnet152(weights=ResNet152_Weights.IMAGENET1K_V2)
for param in model.parameters():
    param.requires_grad = False
# Define the layers you want to add
model.fc = create_dynamic_network(model.fc.in_features, 43, num_layers=num_layers, num_neurons=num_neurons)
model = model.to(device)

# Define loss function, optimizer, etc.
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.fc.parameters(), lr=learning_rate)
scheduler = lr_scheduler.StepLR(optimizer, step_size=step_size, gamma=decay_rate)

### Train the model

In [8]:
# Train model
trained_model = train_model(device=device, model=model, criterion=criterion, optimizer=optimizer, scheduler=scheduler,
                            train_loader=train_loader, num_epochs=num_epochs)

----------
Epoch 1/32
----------


KeyboardInterrupt: 

# Saving the trained model

In [106]:
print('Finished Training')
os.makedirs('./models', exist_ok=True)
PATH = './models/trained_model.pth'
torch.save(trained_model, PATH)

Finished Training


# Loading the model

In [107]:
trained_model = torch.load('./models/trained_model.pth', map_location=device)

# Evaluating the model

In [110]:
def test_model(model, dataloader, device):
    model.eval()  # Set the model to evaluation mode
    running_corrects = 0

    # Disable gradient calculation to speed up the process and reduce memory usage
    with torch.no_grad():
        for i, (images, labels) in enumerate(dataloader):
            images = images.to(device)
            labels = labels.to(device)

            # Forward pass to get output/logits
            outputs = model(images)

            # Get predictions from the maximum value
            _, preds = torch.max(outputs, 1)

            # Increment the correct predictions count
            running_corrects += torch.sum(preds == labels.data)

            # Optionally print progress every 250 batches
            if (i + 1) % 250 == 0:
                print(f'Evaluating: [{i + 1}/{len(dataloader)}],  Correct classified: {running_corrects}/{i + 1}')

    # Calculate the accuracy by dividing the number of correct predictions by the dataset size
    test_acc = running_corrects.double() / len(dataloader)
    print(f'Test Acc: {test_acc:.4f}, Correct classified: {running_corrects}/{len(dataloader)}')

    return test_acc

In [111]:
test_loader = DataLoader(test_dataset, shuffle=True)

test_model(trained_model, test_loader, device)

Evaluating: [250/12630],  Correct classified: 5/250
Evaluating: [500/12630],  Correct classified: 7/500
Evaluating: [750/12630],  Correct classified: 18/750
Evaluating: [1000/12630],  Correct classified: 28/1000
Evaluating: [1250/12630],  Correct classified: 31/1250
Evaluating: [1500/12630],  Correct classified: 38/1500
Evaluating: [1750/12630],  Correct classified: 46/1750
Evaluating: [2000/12630],  Correct classified: 52/2000
Evaluating: [2250/12630],  Correct classified: 60/2250
Evaluating: [2500/12630],  Correct classified: 67/2500
Evaluating: [2750/12630],  Correct classified: 75/2750
Evaluating: [3000/12630],  Correct classified: 80/3000
Evaluating: [3250/12630],  Correct classified: 90/3250
Evaluating: [3500/12630],  Correct classified: 95/3500
Evaluating: [3750/12630],  Correct classified: 102/3750
Evaluating: [4000/12630],  Correct classified: 108/4000
Evaluating: [4250/12630],  Correct classified: 114/4250
Evaluating: [4500/12630],  Correct classified: 121/4500
Evaluating: [4

tensor(0.0258, device='cuda:0', dtype=torch.float64)